All quotes below are excerpted from https://toadstyle.org/cryptopals/59.txt

### Terminology

> An elliptic curve E is just an equation like this:
> y^2 = x^3 + ax + b

> We'll use the notation GF(p) to talk about a finite field of size p. (The "GF" is for "Galois field", another name for a finite field.) When we take a curve E over field GF(p) (written E(GF(p))), what we're saying is that only points with both x and y in GF(p) are valid.

> For example, (3, 6) might be a valid point in E(GF(7)), but it wouldn't be a valid point in E(GF(5)); 6 is not a member of GF(5).

### Group Definitions

> We combine elliptic curve points by adding them.
>
> On an elliptic curve, we define the identity O as an abstract "point at infinity" that doesn't map to any actual (x, y) pair. This might feel like a bit of a hack, but it works. We have the straightforward rule that P + O = P for all P.

> Inversion is way easier in elliptic curves. Just flip the sign on y, and remember that we're in GF(p): invert((x, y)) = (x, -y) = (x, p-y)

> Just like with multiplicative inverses, we have this rule on elliptic curves: P + (-P) = P + invert(P) = O

Let's write a generic Curve implementation with a zero element, inversion, addition, and scalar multiplication:

In [1]:
from challenge_31 import do_sha256, hmac
from challenge_39 import invmod
from challenge_57 import get_small_non_repeated_factors, get_residues
from itertools import count
from operator import mul
from random import randrange
from math import log

In [2]:
class Zero:
    pass


class Curve:
    def __init__(self, a, b, p):
        self.zero = Zero()
        self.a = a
        self.b = b
        self.p = p
    
    def inv(self, pt):
        x, y = pt
        p = self.p
        return (x, p-y)  # = (x, -y) in GF(p)

    def add(self, p1, p2):  # don't worry about how this works. it's ~magic~
        zero = self.zero
        if p1 is zero: return p2
        if p2 is zero: return p1
        if p1 == self.inv(p2): return zero
        
        a, p = self.a, self.p
        x1, y1 = p1
        x2, y2 = p2
        
        if p1 == p2:
            top = (3 * x1**2 + a) % p
            btm = (2 * y1) % p
        else:
            top = (y2 - y1) % p
            btm = (x2 - x1) % p
        m = (top * invmod(btm, p)) % p
        
        x3 = (m**2 - x1 - x2) % p
        y3 = (m*(x1 - x3) - y1) % p
        return x3, y3
    
    def mul(self, pt, k):
        result = self.zero
        add = self.add
        while k:
            if k & 1:
                result = add(result, pt)
            pt = add(pt, pt)
            k >>= 1
        return result

`cryptopals/59.txt` goes on to present generic implementations for scalar multiplication and Diffie-Hellman in order to prove that the algorithms really are generic to any group.

We could write generic versions of `inv`, `add`, `mul` that would take either ints or EC points... we could, but we won't, for the same reason we're using `Curve.add` instead of defining a `CurvePoint` dataclass and giving it an `__add__` method: because that sort of unnecessary complexity would distract from what's important here.

To be specific, what's important is this:

> The fact that these two settings share so many similarities (and can
even share a naive implementation) is great news. It means we already
have a lot of the tools we need to reason about (and attack) elliptic
curves!
>
> Let's put this newfound knowledge into action. Implement a set of
functions up to and including elliptic curve scalar
multiplication. (Remember that all computations are in GF(p), i.e. mod
p.) You can use this curve:
>
>    y^2 = x^3 - 95051*x + 11279326
>
> Over GF(233970423115425145524320034830162017933). Use this base point:
>
>    (182, 85518893674295321206118380980485522083)
>
> It has order 29246302889428143187362802287225875743.
>
> Oh yeah, order. Finding the order of an elliptic curve group turns out
to be a bit tricky, so just trust me when I tell you this one has
order 233970423115425145498902418297807005944. That factors to 2^3 *
29246302889428143187362802287225875743.

In [3]:
# Let's instantiate this curve.
curve = Curve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)

base = (182, 85518893674295321206118380980485522083)
order = 29246302889428143187362802287225875743

# Quick test: make sure the base point times the order equals the group's identity element
assert curve.mul(base, order) is curve.zero

OK, now we're going to implement Diffie-Hellman on elliptic curve groups.

> Implement ECDH and verify that you can do a handshake correctly. In
this case, Alice and Bob's secrets will be scalars modulo the base
point order and their public elements will be points. If you
implemented the primitives correctly, everything should "just work".


To do this, we'll write a class representing a single ECDH keypair. It will need to have a method for generating the keypair and a method for combining a (local) private key with a (remote) public key to get a shared secret.

In [4]:
class ECDHKeypair:
    _priv = None
    pub = None
    
    def __init__(self, curve):
        self.curve = curve
        self.keygen()
        
    def keygen(self):
        curve = self.curve
        
        priv = randrange(0, order)
        pub = curve.mul(base, priv)
        
        self._priv = priv
        self.pub = pub
    
    def handshake(self, other_pub):
        return self.curve.mul(other_pub, self._priv)

In [5]:
# Let's run through a test handshake to make sure our ECDH implementation is sound.
# We'll encapsulate this test in a function to avoid polluting the top-level namespace.

def test_handshake():
    alice = ECDHKeypair(curve)
    bob = ECDHKeypair(curve)

    alice_secret = alice.handshake(bob.pub)
    bob_secret = bob.handshake(alice.pub)

    print("Alice's version of shared secret:", alice_secret)
    print("Bob's version of shared secret:  ", bob_secret)
    assert alice_secret == bob_secret
    print("ECDH handshake successful!")
    
test_handshake()

Alice's version of shared secret: (121145015359528303243721541065792101211, 119815507048302653652519357467812339207)
Bob's version of shared secret:   (121145015359528303243721541065792101211, 119815507048302653652519357467812339207)
ECDH handshake successful!


> Next, reconfigure your protocol from #57 to use it.

(meaning to use ECDH)

In challenge 57 we implemented Bob as a coroutine, and we'll do the same thing here. This poor hapless sap will:

1. Generate a local keypair and share its public key
2. Receive a public key (i.e. curve point)
3. Generate a shared secret
4. Compute `HMAC(message, secret)` for some fixed message
5. Assemble a 2-tuple `(message, mac)` to be yielded
6. `GOTO 2`

In [6]:
def point_to_bytes(pt):
    # helper function
    # this is a quick & dirty hack for turning an EC point into something we can hash
    # assumes log(p, 2) < 128
    return pt[0].to_bytes(128, 'big') + pt[1].to_bytes(128, 'big')
assert log(curve.p, 2) < 128


def bob_coro(message=b'crazy flamboyant for the rap enjoyment', curve=curve):
    keypair = ECDHKeypair(curve)

    # announce our public key on coroutine initialization (before generating first response)
    output = keypair.pub
    
    while True:
        remote_pub = (yield output)
        secret = keypair.handshake(remote_pub)
        mac_key = do_sha256(point_to_bytes(secret))
        mac = hmac(mac_key, message)
        output = (message, mac)

In [7]:
# quick test: make sure Bob gives us correct MACs and doesn't throw any errors

def test_bob(n=10):
    bob = bob_coro()
    bob_pub = next(bob)

    for _ in range(n):
        keypair = ECDHKeypair(curve)
        message, mac = bob.send(keypair.pub)

        mac_key = do_sha256(point_to_bytes(keypair.handshake(bob_pub)))
        assert hmac(mac_key, message) == mac

    print("Bob appears to be working!")
    
test_bob()

Bob appears to be working!


OK, finally we're ready to get to the attack! Here's the end of `cryptopals/59.txt`:

> Can we apply the subgroup-confinement attacks from #57 in this
setting? At first blush, it seems like it will be pretty difficult,
since the cofactor is so small. We can recover, like, three bits by
sending a point with order 8, but that's about it. There just aren't
enough small-order points on the curve.
>
> How about not on the curve?
>
> Wait, what? Yeah, points *not* on the curve. Look closer at our
combine function. Notice anything missing? The b parameter of the
curve is not accounted for anywhere. This is because we have four
inputs to the calculation: the curve parameters (a, b) and the point
coordinates (x, y). Given any three, you can calculate the fourth. In
other words, we don't need b because b is already baked into every
valid (x, y) pair.
>
> There's a dangerous assumption there: namely, that the peer will
submit a valid (x, y) pair. If Eve can submit an invalid pair, that
really opens up her play: now she can pick points from any curve that
differs only in its b parameter. All she has to do is find some curves
with small subgroups and cherry-pick a few points of small
order. Alice will unwittingly compute the shared secret on the wrong
curve and leak a few bits of her private key in the process.
>
> How do we find suitable curves? Well, remember that I mentioned
counting points on elliptic curves is tricky. If you're very brave,
you can implement Schoof-Elkies-Atkins. Or you can use a computer
algebra system like SageMath. Or you can just use these curves I
generated for you:
>
> y^2 = x^3 - 95051*x + 210
>
> y^2 = x^3 - 95051*x + 504
>
> y^2 = x^3 - 95051*x + 727
>
> They have orders:
>
> 233970423115425145550826547352470124412
>
> 233970423115425145544350131142039591210
>
> 233970423115425145545378039958152057148
>
> They should have a fair few small factors between them. So: find some
points of small order and send them to Alice. You can use the same
trick from before to find points of some prime order r. Suppose the
group has order q. Pick some random point and multiply by q/r. If you
land on the identity, start over.
>
> It might not be immediately obvious how to choose random points, but
you can just pick an x and calculate y. This will require you to
implement a modular square root algorithm; use Tonelli-Shanks, it's
pretty straightforward.
>
> Implement the key-recovery attack from #57 using small-order points
from invalid curves.

We'll start by implementing some helper functions, then we'll load in these new curves, factor their orders, and launch the attack.

In [8]:
class NoQuadraticResidueError(Exception):  pass


def eulers_criterion(n, p):
    # tests whether n is a quadratic residue mod p
    # (i.e. whether there exists x such that pow(x, 2, p) == n)
    return pow(n, (p-1)//2, p) == 1


def tonelli_shanks(n, p):
    if not eulers_criterion(n, p):
        raise NoQuadraticResidueError

    # ref: https://en.wikipedia.org/wiki/Tonelli%E2%80%93Shanks_algorithm

    # 1. find Q, S such that Q is odd and Q * 2**S = p-1
    Q, S = p-1, 0
    while Q & 1 == 0:  # faster than Q % 2 == 0
        Q >>= 1  # faster than Q //= 2
        S += 1

    # 2. find some int z such that z is not a quadratic residue mod p
    z = 2
    while eulers_criterion(z, p):
        z += 1
        assert z < p

    # 3. initialize main loop's state variables
    M = S
    c = pow(z, Q, p)
    t = pow(n, Q, p)
    R = pow(n, (Q+1) // 2, p)

    # 4. main loop
    while t > 1:
        # find i's value using repeated squaring
        t_sq = t
        for i in count(1):
            t_sq = pow(t_sq, 2, p)
            if t_sq == 1:
                break
            assert i < M  # sanity-check: if i >= M the residue doesn't exist
                          # (shouldn't ever happen - we made sure the residue
                          # exists back at the top of this function)

        # update state variables and loop
        exponent = M - i - 1
        if exponent < 0:
            b = pow(c, 2**(-exponent), p)
            b = (b * invmod(c, p)) % p
        else:
            b = pow(c, 2**exponent, p)

        M = i
        c = pow(b, 2, p)
        t = (t * c) % p
        R = (R * b) % p

    if t == 0:
        return 0
    
    res1 = R
    res2 = (-R) % p
    return res1, res2

In [9]:
def test_tonelli_shanks():
    p = 17
    for i in range(1, p):
        sq = pow(i, 2, p)
        roots = tonelli_shanks(sq, p)
        assert i in roots
    print("Tonelli-Shanks appears to be working!")
test_tonelli_shanks()

Tonelli-Shanks appears to be working!


In [10]:
def find_point_of_order_r(r, curve, curve_order):
    a, b, p, zero = curve.a, curve.b, curve.p, curve.zero

    while True:
        # generate a random point
        x = randrange(0, p)
        
        # plug x into the curve eqn to find y^2
        rhs = (pow(x, 3, p) + a*x + b) % p
        
        # go from y^2 to y
        try:
            y = tonelli_shanks(rhs, p)[0]  # arbitrarily use the 1st residue returned by t_s
        except NoQuadraticResidueError:
            continue
        pt = (x, y)

        # test whether pt has order r; if so, return pt
        pt2 = curve.mul(pt, curve_order // r)
        if pt2 is not zero:
            print(pt2)
            assert curve.mul(pt2, r) is zero
            return pt2

In [11]:
# Parameters for our new curves:
b_vals = [210, 504, 727]
new_orders = [233970423115425145550826547352470124412,
              233970423115425145544350131142039591210,
              233970423115425145545378039958152057148]

In [13]:
moduli = []
residues = []

for b_val, new_order in zip(b_vals, new_orders):
    new_curve = Curve(curve.a, b_val, curve.p)
    
    print("\nNow using b =", b_val)
    print("Partially factoring curve's order...")
    
    divisors = [d for d in get_small_non_repeated_factors(new_order) if d not in moduli]
    moduli += divisors
    
    if divisors:
        print("New moduli:", divisors)
        print("Gathering residues...")
        for d in divisors:
            find_point_of_order_r(d, new_curve, new_order)
            # ... TODO


Now using b = 210
Partially factoring curve's order...
New moduli: [3, 11, 23, 31, 89, 4999, 28411, 45361]
Gathering residues...
(105895660736863454274321881122632654157, 342531483552213517225306629834659424)
(213412672914991099142506302963193562406, 207149168213502609643003081041108305186)
(185707704851503532345203691135033838236, 65720105455130162260752031947584567858)
(26675675618180228616161875539640372341, 224152552933950816688987311025950690252)
(11219099304956698015468589612844233090, 199721134676338249818087241328268607044)
(114997849209460030518299448593245015146, 133386745956653297588407803279467747610)
(134824467326667956934949377741587406677, 180909713851137550917530922290098748474)
(65830896866488130854547356477734124013, 169356119338825944152247951386978625457)

Now using b = 504
Partially factoring curve's order...
New moduli: [2, 5, 7, 61, 12157, 34693]
Gathering residues...
(88667665663832987305896363298390924083, 0)


InvModException: no inverse